<a href="https://colab.research.google.com/github/ShauryaPundirGraphicEra/OralVis-AI-Research-Intern-Task/blob/main/AI_Intern_Task_Submission_by_Shaurya_Pundir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The dataset should now be downloaded to the `/content/dataset` directory. You can inspect the contents of this directory using the file explorer in the left sidebar or by running shell commands like `!ls /content/dataset`.

## List files and pair images with labels

### Subtask:
Get the list of image files and their corresponding label files.


**Reasoning**:
Define the paths to the image and label directories, list the image files, construct the corresponding label file paths, and create a list of tuples containing image and label file paths.



In [2]:
import os

images_dir = '/content/drive/MyDrive/ToothNumber_TaskDataset/images'
labels_dir = '/content/drive/MyDrive/ToothNumber_TaskDataset/labels'

image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]

image_label_pairs = []
for image_file in image_files:
    # Assuming label files have the same name but with a .txt extension
    label_file_name = os.path.splitext(image_file)[0] + '.txt'
    label_file_path = os.path.join(labels_dir, label_file_name)

    # Check if the corresponding label file exists before adding the pair
    if os.path.exists(label_file_path):
        image_label_pairs.append((os.path.join(images_dir, image_file), label_file_path))

print(f"Found {len(image_label_pairs)} image-label pairs.")

Found 497 image-label pairs.


In [3]:
from sklearn.model_selection import train_test_split

# First split: 80% train, 20% temporary (validation + test)
train_pairs, temp_pairs = train_test_split(image_label_pairs, train_size=0.8, random_state=42)

# Second split: split the temporary set into 50% validation and 50% test (10% each of original)
val_pairs, test_pairs = train_test_split(temp_pairs, train_size=0.5, random_state=42)

print(f"Training set size: {len(train_pairs)}")
print(f"Validation set size: {len(val_pairs)}")
print(f"Testing set size: {len(test_pairs)}")

Training set size: 397
Validation set size: 50
Testing set size: 50


In [4]:
import shutil

# Define the base output directory
output_dir = '/content/drive/MyDrive/final_dataset'

# Create the directory structure
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_dir, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, split, 'labels'), exist_ok=True)

# Function to copy files
def copy_pairs(pairs, split):
    for img_path, lbl_path in pairs:
        shutil.copy(img_path, os.path.join(output_dir, split, 'images'))
        shutil.copy(lbl_path, os.path.join(output_dir, split, 'labels'))

# Copy data into folders
copy_pairs(train_pairs, 'train')
copy_pairs(val_pairs, 'val')
copy_pairs(test_pairs, 'test')

print("✅ Dataset successfully organized into final_dataset/")
print(f"Train: {len(train_pairs)}, Val: {len(val_pairs)}, Test: {len(test_pairs)}")


✅ Dataset successfully organized into final_dataset/
Train: 397, Val: 50, Test: 50


In [7]:
!git clone https://github.com/yolo-org/yolov11.git
!cd yolov11
!pip install -r requirements.txt


Cloning into 'yolov11'...
fatal: could not read Username for 'https://github.com': No such device or address
/bin/bash: line 1: cd: yolov11: No such file or directory
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [11]:
!pip install ultralytics -U

In [15]:
!wget https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8m.pt -O yolov8m.pt

--2025-08-29 12:05:37--  https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8m.pt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/521807533/8e15b10b-c088-48cd-835d-458343ff2742?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-29T12%3A59%3A49Z&rscd=attachment%3B+filename%3Dyolov8m.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-29T11%3A58%3A56Z&ske=2025-08-29T12%3A59%3A49Z&sks=b&skv=2018-11-09&sig=eDxTUmISOsg2IuFMU%2FHHuGe0QGIu23EXP5dQW46W6cY%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NjQ2OTQzNywibmJmIjoxNzU2NDY5MTM3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmN

In [18]:
from ultralytics import YOLO

# Initialize model with pretrained medium variant
model = YOLO("yolov8m.pt")  # Change to correct filename for medium variant

# Stage 1: Freeze early layers
model.train(
    data="/content/data.yaml",
    epochs=10,
    freeze=[0, 1, 2, 3, 4],
    lr0=1e-3,
    batch=16
)

# Stage 2: Unfreeze all layers for fine-tuning
model.train(
    data="/content/data.yaml",
    epochs=40,
    lr0=1e-4,
    batch=16
)


Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=[0, 1, 2, 3, 4], half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrai

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a5f40dc5ca0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,

In [21]:
model = YOLO("/content/runs/detect/train32/weights/best.pt")  # Use best.pt from epoch 40

# Continue training
model.train(
    data="/content/data.yaml",  # Correct data path
    epochs=30,
    lr0=1e-5,   # Lower learning rate
    batch=16,
    imgsz=640   # Match input size from previous runs
)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=1e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/detect/train32/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a5f40f5dc70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,

In [24]:
# Cell 8: Stage 4 - Additional Fine-Tuning

model=YOLO("/content/runs/detect/train4/weights/best.pt")

model.train(
    data="/content/data.yaml",
    epochs=20,  # Adjust to 10-15
    lr0=1e-6,   # Very low learning rate
    batch=16
)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=1e-06, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/detect/train4/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=Tr

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a6041980fe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,

In [27]:
# Cell 7: Evaluate on validation set
metrics_val = model.val()
print("Validation Metrics:", metrics_val.box.map, metrics_val.box.map50, metrics_val.box.map75)

# Cell 8: Evaluate on test set (if specified in dataset.yaml)
metrics_test = model.val(split='test')
print("Test Metrics:", metrics_test.box.map, metrics_test.box.map50, metrics_test.box.map75)

# Cell 9: Generate sample predictions (3 test images)
test_dir = "/content/drive/MyDrive/final_dataset/test/images"  # Adjust path as per your dataset.yaml
test_images = [os.path.join(test_dir, img) for img in os.listdir(test_dir)][:3]
pred_results = model.predict(test_images, save=True, save_txt=True)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.6±0.6 ms, read: 17.1±7.0 MB/s, size: 74.2 KB)
val: Scanning /content/drive/MyDrive/final_dataset/val/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 50/50 45669.7it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.73it/s 5.5s
                   all         50       1435      0.921      0.916      0.959      0.803
           Canine (13)         47         47      0.925      0.915      0.976       0.79
           Canine (23)         47         47      0.935      0.924      0.973      0.799
           Canine (33)         50         50      0.901      0.907      0.944      0.787
           Canine (43)         49         50      0.937      0.897      0.925       0.75
  Central Incisor (21)         47         47      0.932      0.936       0.96      0.812
  Central Incisor (41)      

In [28]:
import shutil
import os
from google.colab import files

# Zip the directories
shutil.make_archive('/content/train65', 'zip', '/content/runs/detect/train65')
shutil.make_archive('/content/train66', 'zip', '/content/runs/detect/train66')

# Download the zip files
files.download('/content/train65.zip')
files.download('/content/train66.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>